In [1]:
import polars as pl

from src import constants

dataset = pl.scan_csv(constants.OUTPUT_DIR / "dataset.csv").collect()
dataset

session_id,yad_no,session_length,last_seen_yad_no,last_seen_sml_cd_0,last_seen_lrg_cd_0,last_seen_ken_cd_0,first_seen_yad_no,first_seen_sml_cd_0,first_seen_lrg_cd_0,first_seen_ken_cd_0,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd,total_room_cnt_null_flg,wireless_lan_flg_null_flg,kd_stn_5min_null_flg,kd_bch_5min_null_flg,kd_slp_5min_null_flg,kd_conv_walk_5min_null_flg,mean_total_room_cnt_with_onsen_flg,std_total_room_cnt_with_onsen_flg,median_total_room_cnt_with_onsen_flg,mean_total_room_cnt_with_yad_type,std_total_room_cnt_with_yad_type,median_total_room_cnt_with_yad_type,mean_total_room_cnt_with_kd_bch_5min,std_total_room_cnt_with_kd_bch_5min,…,top3_sml_cd_yad_no,top4_sml_cd_yad_no,top5_sml_cd_yad_no,top6_sml_cd_yad_no,top7_sml_cd_yad_no,top8_sml_cd_yad_no,top9_sml_cd_yad_no,top0_lrg_cd_yad_no,top1_lrg_cd_yad_no,top2_lrg_cd_yad_no,top3_lrg_cd_yad_no,top4_lrg_cd_yad_no,top5_lrg_cd_yad_no,top6_lrg_cd_yad_no,top7_lrg_cd_yad_no,top8_lrg_cd_yad_no,top9_lrg_cd_yad_no,top0_ken_cd_yad_no,top1_ken_cd_yad_no,top2_ken_cd_yad_no,top3_ken_cd_yad_no,top4_ken_cd_yad_no,top5_ken_cd_yad_no,top6_ken_cd_yad_no,top7_ken_cd_yad_no,top8_ken_cd_yad_no,top9_ken_cd_yad_no,top0_wid_cd_yad_no,top1_wid_cd_yad_no,top2_wid_cd_yad_no,top3_wid_cd_yad_no,top4_wid_cd_yad_no,top5_wid_cd_yad_no,top6_wid_cd_yad_no,top7_wid_cd_yad_no,top8_wid_cd_yad_no,top9_wid_cd_yad_no
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""85aeeda68256d0…",11037,3,7947,442,278,2,7947,442,278,2,0,603.0,1.0,0,1.0,1.0,1.0,1.0,3,41,180,191,0,0,0,1,1,0,107.705414,119.32106,78.0,87.9898,107.357445,52.0,48.576614,81.2795,…,12089,12986,10155,7379,12132,8468,6905,12425,6199,12240,11037,2087,12089,12986,10155,7379,12132,12425,6199,8445,12240,11037,9104,3077,2087,12089,12986,10095,3338,12350,8553,385,3848,13468,12785,13549,755
"""dd2bf842603d6c…",8567,1,12524,13,137,20,12524,13,137,20,0,641.0,1.0,1,1.0,1.0,1.0,1.0,11,2,278,442,0,0,0,1,1,0,52.679703,69.28917,29.0,87.9898,107.357445,52.0,48.576614,81.2795,…,5116,12017,6693,6470,2797,2160,2687,719,8567,2201,5116,12017,6693,6470,2797,1587,2160,13017,719,1818,8567,9971,2201,693,3988,6418,12619,13017,719,1818,8567,9971,2201,693,3988,1799,6418
"""89f36f3cadfe6a…",915,1,8170,11,290,41,8170,11,290,41,0,428.0,1.0,0,1.0,1.0,1.0,1.0,3,18,153,69,0,0,1,1,1,1,107.705414,119.32106,78.0,87.9898,107.357445,52.0,48.576614,81.2795,…,3848,13468,1091,532,915,4913,7093,10095,3338,12350,8553,3848,13468,1091,532,915,4913,10095,3338,12350,8553,3848,13468,1091,532,915,4913,10095,3338,12350,8553,385,3848,13468,12785,13549,755
"""6fa5d93c75be59…",6470,2,3848,69,153,18,3848,69,153,18,0,390.0,1.0,0,1.0,1.0,1.0,1.0,11,2,278,442,0,0,0,1,1,0,107.705414,119.32106,78.0,87.9898,107.357445,52.0,48.576614,81.2795,…,5116,12017,6693,6470,2797,2160,2687,719,8567,2201,5116,12017,6693,6470,2797,1587,2160,13017,719,1818,8567,9971,2201,693,3988,6418,12619,13017,719,1818,8567,9971,2201,693,3988,1799,6418
"""6b78efe3d4394b…",9971,2,9981,336,62,26,9981,336,62,26,0,466.0,1.0,0,1.0,1.0,1.0,1.0,11,2,74,226,0,0,0,1,1,0,107.705414,119.32106,78.0,87.9898,107.357445,52.0,48.576614,81.2795,…,2041,6125,4763,13647,6431,13754,4221,13017,1818,9971,693,3988,12619,4646,2739,9149,13463,13017,719,1818,8567,9971,2201,693,3988,6418,12619,13017,719,1818,8567,9971,2201,693,3988,1799,6418
"""c3b08fb741a976…",6199,5,3848,69,153,18,3848,69,153,18,0,480.0,1.0,0,1.0,1.0,1.0,1.0,3,41,180,191,0,0,0,1,1,0,107.705414,119.32106,78.0,87.9898,107.357445,52.0,48.576614,81.2795,…,12089,12986,10155,7379,12132,8468,6905,12425,6199,12240,11037,2087,12089,12986,10155,7379,12132,12425,6199,8445,12240,11037,9104,3077,2087,12089,12986,10095,3338,12350,8553,385,3848,13468,12785,13549,755
"""021eb5210

In [2]:
dataset.group_by("session_id").first().shape[0]

174700

In [3]:
preds = pl.scan_csv(constants.OUTPUT_DIR / "exp000" / "cv_fold0.csv").collect()
preds

session_id,yad_no,predict
str,i64,i64
"""6d1633e4185e67…",1013,1013
"""6d1633e4185e67…",1013,1424
"""6d1633e4185e67…",1013,5203
"""6d1633e4185e67…",1013,11450
"""6d1633e4185e67…",1013,9405
"""6d1633e4185e67…",1013,4424
"""6d1633e4185e67…",1013,8579
"""6d1633e4185e67…",1013,5357
"""6d1633e4185e67…",1013,3367


In [ ]:
num_session_has_label_in_predict =(
    preds
    .group_by("session_id")
    .map_groups(
        lambda group: group.with_columns(pl.arange(0, group.height).alias("seq_no"))
    )
).filter(pl.col("yad_no") == pl.col("predict"))["session_id"].n_unique()

In [ ]:
num_session_has_label_in_predict / preds["session_id"].n_unique()

0.477753338298204

In [ ]:
preds["session_id"].unique().to_list()[:3]

['8e74132684c7b9a55b2769c454ca082c',
 '3eb36287a40827ccb59e89067c71d30e',
 'f9eff605f0b1b0acdb6c4b98dd83b1b8']

In [ ]:
train_label_df = pl.scan_csv(constants.INPUT_DIR / "train_label.csv").collect()
train_label_df

session_id,yad_no
str,i64
"""000007603d533d…",4101
"""0000ca043ed437…",8253
"""0000d4835cf113…",4863
"""0000fcda1ae1b2…",1652
"""000104bdffaaad…",96
"""00011afe25c343…",4823
"""000125c737df18…",10378
"""0001763050a10b…",10362
"""000178c4d4d567…",1227


In [ ]:
see_targets = [
    "187cbc12d251ad3360f717c1cba7f1f2"
]
for target in see_targets:
    print(target)
    ds_this_target = dataset.filter(pl.col("session_id") == target)
    preds_this_target = preds.filter(pl.col("session_id") == target)
    print("DS: ", ds_this_target)
    print("PREDS: ", preds_this_target)
    yad_candidates = ds_this_target["yad_no"].to_list()
    pred = preds_this_target["predict"].to_list()
    label = train_label_df.filter(pl.col("session_id") == target)["yad_no"].to_list()[0]
    print("YAD CANDIDATES: ", yad_candidates)
    print("PREDS: ", pred)
    print("LABEL: ", label)
    print("Label in candidates: ", label in yad_candidates)
    print("")

187cbc12d251ad3360f717c1cba7f1f2
DS:  shape: (0, 40)
┌────────────┬────────┬────────────┬──────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ session_id ┆ yad_no ┆ session_le ┆ yad_type ┆ … ┆ mean_total ┆ std_total ┆ median_to ┆ diff_tota │
│ ---        ┆ ---    ┆ ngth       ┆ ---      ┆   ┆ _room_cnt_ ┆ _room_cnt ┆ tal_room_ ┆ l_room_cn │
│ str        ┆ i64    ┆ ---        ┆ i64      ┆   ┆ with_kd_co ┆ _with_kd_ ┆ cnt_with_ ┆ t_with_on │
│            ┆        ┆ i64        ┆          ┆   ┆ nv…        ┆ conv_…    ┆ kd_co…    ┆ sen_f…    │
│            ┆        ┆            ┆          ┆   ┆ ---        ┆ ---       ┆ ---       ┆ ---       │
│            ┆        ┆            ┆          ┆   ┆ f64        ┆ f64       ┆ f64       ┆ f64       │
╞════════════╪════════╪════════════╪══════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
└────────────┴────────┴────────────┴──────────┴───┴────────────┴───────────┴───────────┴───────────┘
PREDS:  shape: (0, 3)
┌────────────┬──

In [ ]:
raise Exception()

from tqdm.auto import tqdm

label_in_candidates = 0
unique_session = dataset["session_id"].unique().to_list()
for target in tqdm(unique_session, total=len(unique_session)):
    ds_this_target = dataset.filter(pl.col("session_id") == target)
    preds_this_target = preds.filter(pl.col("session_id") == target)
    yad_candidates = ds_this_target["yad_no"].to_list()
    pred = preds_this_target["predict"].to_list()
    label = train_label_df.filter(pl.col("session_id") == target)["yad_no"].to_list()[0]
    if label in yad_candidates:
        label_in_candidates += 1

print(label_in_candidates / len(dataset["session_id"].unique().to_list()))

Exception: 